In [1]:
import nltk
import pandas as pd
import numpy as np
import json
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.tokenize import sent_tokenize

# Train Set

In [2]:
json_file = open("C:/Users/61416/Desktop/project-files/train.json", "r", encoding="utf-8")
sh_json= json.load(json_file)
json_file.close()

In [3]:
json_file_ex = open("C:/Users/61416/Desktop/project-files/climate.json", "r", encoding="utf-8")
sh_json_ex= json.load(json_file_ex)
json_file.close()

In [4]:
dic_ex=[]
for i in sh_json_ex:
    dic_ex.append(i)
print(dic_ex[0])
final_ex=[]
label_ex=[]
for i in range(len(dic_ex)):
    final_ex.append(dic_ex[i]["text"])
    label_ex.append(dic_ex[i]["label"])
print(final_ex[0])
print(label_ex[0])

{'text': 'WASHINGTON  —   Congressional Republicans have a new fear when it comes to their    health care lawsuit against the Obama administration: They might win. The incoming Trump administration could choose to no longer defend the executive branch against the suit, which challenges the administration’s authority to spend billions of dollars on health insurance subsidies for   and   Americans, handing House Republicans a big victory on    issues. But a sudden loss of the disputed subsidies could conceivably cause the health care program to implode, leaving millions of people without access to health insurance before Republicans have prepared a replacement. That could lead to chaos in the insurance market and spur a political backlash just as Republicans gain full control of the government. To stave off that outcome, Republicans could find themselves in the awkward position of appropriating huge sums to temporarily prop up the Obama health care law, angering conservative voters who h

In [5]:
dic_in=[]
for i in sh_json:
    dic_in.append(sh_json[i])
print(dic_in[0])
final_in=[]
label_in=[]
for i in range(len(dic_in)):
    final_in.append(dic_in[i]["text"])
    label_in.append(dic_in[i]["label"])
print(final_in[0])
print(label_in[0])

{'text': 'why houston flooding isn‘t a sign of climate change\nDistinguished US climate scientist, Dr Roy Spencer writes: "In the context of climate change, is what we are seeing in Houston a new level of disaster which is becoming more common? The flood disaster unfolding in Houston is certainly very unusual. But so are other natural weather disasters, which have always occurred and always will occur....Major floods are difficult to compare throughout history because of the ways we alter the landscape., For example, as cities like Houston expand over the years, soil is covered up by roads, parking lots and buidings, with water rapidly draining off rather than soaking into the soil. The population of Houston is now ten times what is was in the 1920s. \xa0The Houston metroplex has expanded greatly and the water drainage is basically in the direction of downtown Houston."', 'label': 1}
why houston flooding isn‘t a sign of climate change
Distinguished US climate scientist, Dr Roy Spencer 

In [6]:
final=[]
label=[]
final=final_in+final_ex
label=label_in+label_ex

In [7]:
print(len(label))
print(len(final))

1510
1510


In [8]:
final_lower=[]
for i in final:
    final_lower.append(i.lower())



In [9]:
import string
final_lower_filtered=[]
for i in range(len(final_lower)):
    a=nltk.word_tokenize(final_lower[i])
    words = [word for word in a if word.isalpha()]
    st=" ".join(words)
    final_lower_filtered.append(st)

In [10]:
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()
lem=[]
for i in final_lower_filtered:
    k=[]
    sentence7 = nlp(i)
    for word in sentence7:
        k.append(word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_)
    z=" ".join(k)
    lem.append(z)

In [11]:
print(lem[0])

why houston flooding isn t a sign of climate change distinguish us climate scientist dr roy spencer write in the context of climate change be what we be see in houston a new level of disaster which be become more common the flood disaster unfold in houston be certainly very unusual but so be other natural weather disaster which have always occur and always will occur flood be difficult to compare throughout history because of the way we alter the for example as city like houston expand over the year soil be cover up by road parking lot and buiding with water rapidly drain off rather than soak into the soil the population of houston be now ten time what be be in the the houston metroplex have expand greatly and the water drainage be basically in the direction of downtown houston


In [12]:
modals = ['can', 'could', 'may', 'might', 'must', 'will', 'would', 'should',"couldn't",'could not','occasionally','possibly','perhaps','rarely','sometimes', 'unsure', 'often', 'likely', 'usually' ,'definitely ','regularly' ,'frequent', 'frequently','all'
,'enough','many','little','few','several','major','majorly','greatly','great','number','none','No','some','more','most','lots','less','least',
'any','plenty','study','shows','show','fears','fear','theory','worse','worst','fails','fail']
modals_train=[]
for i in range(len(lem)):
    count=0
    a=nltk.word_tokenize(lem[i])
    for i in a:
        if (i.lower() in modals):
            count+=1
    modals_train.append(count)
print(len(modals_train))


1510


In [13]:
c=["climate","weather","fire","floods","temprature","warming","rain","melt","melting"]
climate_train=[]
for i in range(len(final_lower)):
    count=0
    a=nltk.word_tokenize(final_lower[i])
    for i in a:
        if (i.lower() in c):
            count+=1
    climate_train.append(count)
print((climate_train[0]))

5


In [14]:
c=['?','!','"']
punct=[]
for i in range(len(final_lower)):
    count=0
    a=nltk.word_tokenize(final_lower[i])
    for i in a:
        if (i.lower() in c):
            count+=1
    punct.append(count)


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_df=0.95,min_df=3,stop_words='english',ngram_range=(3,3))
vector=vectorizer.fit_transform(lem)

In [16]:
vector.shape

(1510, 5968)

In [17]:
z=vector.todense()
g=z.tolist()
print(len(g[0]))
for i in range(len(modals_train)):
    g[i].append(modals_train[i])
    g[i].append(climate_train[i])
    g[i].append(punct[i])

5968


In [18]:
import scipy
final_vector=scipy.sparse.csr_matrix(g)
final_vector.shape

(1510, 5971)

# Models 

In [19]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(alpha=0.001,loss="modified_huber", penalty="l2", max_iter=5,random_state=100)
clf.fit(final_vector,label)

C:\Users\61416\Anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


SGDClassifier(alpha=0.001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='modified_huber',
              max_iter=5, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=100, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

# Dev Set

In [20]:
json_file_dev = open("C:/Users/61416/Desktop/project-files/dev.json", "r", encoding="utf-8")
sh_json_dev= json.load(json_file_dev)
json_file_dev.close()
dic_dev=[]
for i in sh_json_dev:
    dic_dev.append(sh_json_dev[i])
print(dic_dev[3])
final_dev=[]
label_dev=[]
for i in range(len(dic_dev)):
    final_dev.append(dic_dev[i]["text"])
    label_dev.append(dic_dev[i]["label"])

{'text': 'Morrison a ‘predatory’ centrist on climate policy with no plans for meaningful emissions reduction, says Swan\nLabor’s federal president Wayne Swan will accuse Scott Morrison of engaging in “predatory centrism” on climate policy by styling himself as the pragmatist between the extremes of climate emergencies and denialism, when the government has no intention of driving meaningful emissions reduction.\nIn a speech to be delivered on Sunday, Swan will argue Labor will only win the decade-long climate wars if it approaches the challenge with “pragmatic policy and ruthless organisation”.\nAccording to speech notes, Swan will say Labor needs to articulate a roadmap for the domestic coal powered industry “which manages its decline”.\n“There must be a strong dialogue between government, industry, and the unions, and operate on the principle that no one gets left behind. We have to work closely with coal-dependent regions and put in a plan so that everyone gets a good future.”\nSwan

In [21]:
final_lower_dev=[]
for i in final_dev:
    final_lower_dev.append(i.lower())

In [22]:
import string
final_lower_filtered_dev=[]
for i in range(len(final_lower_dev)):
    a=nltk.word_tokenize(final_lower_dev[i])
    words = [word for word in a if word.isalpha()]
    st=" ".join(words)
    final_lower_filtered_dev.append(st)

In [23]:
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()
lem_dev=[]
for i in final_lower_filtered_dev:
    k=[]
    sentence7 = nlp(i)
    for word in sentence7:
        k.append(word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_)
    z=" ".join(k)
    lem_dev.append(z)

In [24]:
modals = ['can', 'could', 'may', 'might', 'must', 'will', 'would', 'should',"couldn't",'could not','occasionally','possibly','perhaps','rarely','sometimes', 'unsure', 'often', 'likely', 'usually' ,'definitely ','regularly' ,'frequent', 'frequently','all'
,'enough','many','little','few','several','major','majorly','greatly','great','number','none','No','some','more','most','lots','less','least',
'any','plenty','study','shows','show','fears','fear','theory','worse','worst','fails','fail']
modals_dev=[]
for i in range(len(final_lower_dev)):
    count=0
    a=nltk.word_tokenize(final_lower_dev[i])
    for i in a:
        if (i.lower() in modals):
            count+=1
    modals_dev.append(count)
print(len(modals_dev))

100


In [25]:
c=["climate","weather","fire","floods","temprature","warming","rain","melt","melting"]
climate_dev=[]
for i in range(len(final_lower_dev)):
    count=0
    a=nltk.word_tokenize(final_lower_dev[i])
    for i in a:
        if (i.lower() in c):
            count+=1
    climate_dev.append(count)
print((climate_dev[0]))

24


In [26]:
c=['?','!','"']
punct_dev=[]
for i in range(len(final_lower_dev)):
    count=0
    a=nltk.word_tokenize(final_lower_dev[i])
    for i in a:
        if (i.lower() in c):
            count+=1
    punct_dev.append(count)

In [27]:
vector_dev=vectorizer.transform(lem_dev)

In [28]:
z1=vector_dev.todense()
g1=z1.tolist()
print(len(g1[0]))
for i in range(len(modals_dev)):
    g1[i].append(modals_dev[i])
    g1[i].append(climate_dev[i])
    g1[i].append(punct_dev[i])

5968


In [29]:
import scipy
final_vector_dev=scipy.sparse.csr_matrix(g1)
final_vector_dev.shape

(100, 5971)

# Predictions and Metrics

In [30]:
y_pred=clf.predict(final_vector_dev)
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
score = f1_score(label_dev,y_pred)
print('F1 Score: %.3f' % score)
print(precision_score(label_dev,y_pred))

F1 Score: 0.667
0.5


In [31]:
y_pred1=log.predict(final_vector_dev)
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
score = f1_score(label_dev,y_pred1)
print('F1 Score: %.3f' % score)
print(precision_score(label_dev,y_pred1))

NameError: name 'log' is not defined

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(label_dev,y_pred)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(label_dev, y_pred, labels=[0,1]))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(label_dev, y_pred1, labels=[0,1]))

# Test Set 

In [ ]:
json_file_test = open("C:/Users/61416/Desktop/project-files/test-unlabelled.json", "r", encoding="utf-8")
sh_json_test= json.load(json_file_test)
json_file_test.close()
dic_test=[]
for i in sh_json_test:
    dic_test.append(sh_json_test[i])
print(dic_test[3])
final_test=[]
for i in range(len(dic_test)):
    final_test.append(dic_test[i]["text"])

In [ ]:
final_lower_test=[]
for i in final_test:
    final_lower_test.append(i.lower())

In [ ]:
import string
final_lower_filtered_test=[]
for i in range(len(final_lower_test)):
    a=nltk.word_tokenize(final_lower_test[i])
    words = [word for word in a if word.isalpha()]
    st=" ".join(words)
    final_lower_filtered_test.append(st)

In [ ]:
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()
lem_test=[]
for i in final_lower_filtered_test:
    k=[]
    sentence7 = nlp(i)
    for word in sentence7:
        k.append(word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_)
    z=" ".join(k)
    lem_test.append(z)

In [ ]:
modals = ['can', 'could', 'may', 'might', 'must', 'will', 'would', 'should',"couldn't",'could not','occasionally','possibly','perhaps','rarely','sometimes', 'unsure', 'often', 'likely', 'usually' ,'definitely ','regularly' ,'frequent', 'frequently','all'
,'enough','many','little','few','several','major','majorly','greatly','great','number','none','No','some','more','most','lots','less','least',
'any','plenty','study','shows','show','fears','fear','theory','worse','worst','fails','fail']
modals_test=[]
for i in range(len(final_lower_test)):
    count=0
    a=nltk.word_tokenize(final_lower_test[i])
    for i in a:
        if (i.lower() in modals):
            count+=1
    modals_test.append(count)
print(len(modals_test))

In [ ]:
c=["climate","weather","fire","floods","temprature","warming","rain","melt","melting"]
climate_test=[]
for i in range(len(final_lower_test)):
    count=0
    a=nltk.word_tokenize(final_lower_test[i])
    for i in a:
        if (i.lower() in c):
            count+=1
    climate_test.append(count)
print((climate_test[0]))

In [ ]:
c=['?','!','"']
punct_test=[]
for i in range(len(final_lower_test)):
    count=0
    a=nltk.word_tokenize(final_lower_test[i])
    for i in a:
        if (i.lower() in c):
            count+=1
    punct_test.append(count)

In [ ]:
vector_test=vectorizer.transform(lem_test)

In [ ]:
z2=vector_test.todense()
g2=z2.tolist()
print(len(g2[0]))
for i in range(len(modals_test)):
    g2[i].append(modals_test[i])
    g2[i].append(climate_test[i])
    g2[i].append(punct_test[i])

In [ ]:
import scipy
final_vector_test=scipy.sparse.csr_matrix(g2)
final_vector_test.shape

In [ ]:
y_pred_test=clf.predict(final_vector_test)

In [ ]:
list1=[]
for i in y_pred_test:
    list1.append(i)
print(list1)
list2=[]
for i in range(len(sh_json_test)):
    final_dic={}
    final_dic['label']=int(list1[i])
    list2.append(final_dic)
g={}
for i in range(len(sh_json_test)):
    g['test-'+str(i)]=list2[i]
print(g)
with open("test-output.json", "w") as fp:
    json.dump(g , fp) 